<a href="https://colab.research.google.com/github/franklin-univ-data-science/comp411/blob/master/Module06_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 7



1. Explain the idea and calculation steps of PCA.
2. Given the following data set, how many percent of the total variance can be explained by the top 2 principal components?

```python
from sklearn.datasets.samples_generator import make_blobs
from pandas import DataFrame
# generate a classification dataset
X, y = make_blobs(n_samples=1000, centers=3, n_features=10, random_state=1, cluster_std=3)
```

3. What is the most significant difference between PCA and LDA?
4. Based on the data set given above, split data into 80% training and 20% test with stratification, apply PCA to pick the top 2 components, apply a classification algorithm, print the number of misclassified samples in the test set.
5. Apply LDA on the above problem.
6. In two to three paragraphs of prose (i.e. sentences, not bullet lists), summarize and interact with the content that was covered this week in readings and in class meeting. In your summary, you should highlight the major topics, methods, and practices that were covered. Your summary should also interact with the material through personal observations, reflections, and applications to the field of study. In particular, highlight what surprised, enlightened, or otherwise engaged you. In other words, you should think and write critically not just about what was presented but also what you have learned through the session. 


The idea of PCA is that patterns in data can be identified on the correlation between features. The first step is to standardize the d-dimensional dataset by using the mean and standard deviation of each feature. The next step is to construct a covariance matrix which is comparing two variables to measure the change. The third step is to decompose the covariance matrix into eigenvectors and eigenvalues which is the principal components and their magnitude. The next step is to select the k eigenvectors which will correspond with the k largest eigenvalues, using k as the dimensionality of the new feature subspace. After this, construct a projection matrix W from the top k eigenvectors. The final step is to transform the d-dimensional input dataset X using the projection matric W to be able to get the k-dimensional feature subspace.

Using the dataset we can determine that the percent of total variance explained by the top two principal components would be 30 percent.

The main difference between PCA and LDA is their focus for the dataset. PCA focuses on being able to capture the direction of maximum variance for the dataset. LDA focuses on finding the feature subspace that can maximize the separability between the groups of data.

In [1]:
import pandas as pd
from sklearn.datasets import make_blobs
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA


# generate a classification dataset
X, y = make_blobs(n_samples=1000, centers=3, n_features=10, random_state=1, cluster_std=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)



sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

cov_mat = np.cov(X_train_std.T) 

eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)

tot = sum(eigen_vals) 
var_exp = [(i / tot) for i in sorted(eigen_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

eigen_pairs = [(eigen_vals[i], eigen_vecs[:, i]) for i in range(len(eigen_vals))]
eigen_pairs.sort(key=lambda k: k[0], reverse=True)
w = np.array([eigen_pairs[0][1], eigen_pairs[1][1]]).T

pca = PCA()
X_train_pca = pca.fit_transform(X_train_std)

pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

lr = LogisticRegression(solver = 'lbfgs', multi_class='auto')
lr = lr.fit(X_train_pca, y_train)

y_pred = lr.predict(X_test_pca)
print('Misclassified samples: %d' % (y_test != y_pred).sum())

Misclassified samples: 2


In [2]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components=2)
X_train_lda = lda.fit_transform(X_train_std, y_train)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs', multi_class='auto')
lr = lr.fit(X_train_lda, y_train)

X_test_lda = lda.transform(X_test_std)

This week we looked at more data compression and how to achieve this with dimensionality reduction. We started by learning the principal component analysis which helps find patterns within the data and the correlation between their features. PCA is made up of seven steps that end with determining the k-dimensional feature subspace. After this section we looked at the supervised data compression using linear discriminant analysis which looks at the separability between the classes. Each of the data compression types have their own way of using the data. I can see how each could be used and how some one may prefer one over the other.